In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.compat.v1 import keras as K
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import applications
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define the root directory containing student subdirectories
root_directory = "C:/Users/HP/OneDrive/Documents/Photos"

# Initialize lists to store data
data = []
labels = []
common_size = (224, 224)  # Adjust the size as needed

In [ ]:
# Iterate through each subdirectory (student)
for student_name in os.listdir(root_directory):
    student_directory = os.path.join(root_directory, student_name)
    
    # Check if it's a directory
    if os.path.isdir(student_directory):
        # Iterate through each image file in the student's directory
        for image_file in os.listdir(student_directory):
            image_path = os.path.join(student_directory, image_file)
            
            # Read the image using OpenCV
            img = cv2.imread(image_path)
            img_resized = cv2.resize(img, common_size)
            
            # Append the resized image and label to the data lists
            data.append(img_resized)
            labels.append(student_name)

# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

In [ ]:
# Print the shape of the data and labels arrays
print("Data shape:", data.shape)
print("Labels shape:", labels.shape)
print(np.unique(labels))
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

labels_encoded

In [ ]:
# Use data augmentation
datagen_train = ImageDataGenerator(
    rotation_range=20,       
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Split the dataset into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42, stratify=labels)


In [ ]:
# Print the shape of the training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
base_model = applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  # Change 1 to 3 for RGB
# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(BatchNormalization())
model.add(Dense(len(np.unique(labels)), activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [ ]:
# Train the model with data augmentation
# Convert string labels to numerical format
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
train_generator = datagen_train.flow(X_train, y_train_encoded, batch_size=64)


In [ ]:
history = model.fit(train_generator, epochs=50, validation_data=(X_test, y_test_encoded), callbacks=[early_stopping])


In [ ]:
# Evaluate the model on the test set using data generator
test_generator = datagen_train.flow(X_test, y_test_encoded, batch_size=64)
accuracy = model.evaluate(test_generator)[1]
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
# Plot the training and validation loss over epochs
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Create a face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


# Initialize attendance_df outside the function
attendance_df = pd.DataFrame(columns=['Name', 'Time', 'Date'])

# Initialize a buffer for smoothing predicted labels
label_buffer = []

# Initialize temp as a global variable
global temp
temp = 0

def mark_attendance(predictions):
    global attendance_df, temp
    
    # Check if the label buffer is empty
    if not label_buffer:
        return

    # Apply label smoothing and get the final prediction
    final_prediction = max(set(label_buffer), key=label_buffer.count)
    print(f"final prediction: {final_prediction}")
    # Use the mode of the labels in the buffer as the final prediction
    if final_prediction in attendance_df['Name'].values:
        return

    # Check if the maximum confidence is above the threshold
    confidence_threshold = 0.5  # Set your confidence threshold
    if np.max(predictions) > confidence_threshold:
        temp += 1
        # Add to attendance DataFrame
        now = datetime.now()
        current_time = now.strftime('%I:%M:%S %p')
        current_date = now.strftime('%d-%B-%Y')
        new_record = pd.DataFrame({'Name': [predicted_name], 'Time': [current_time], 'Date': [current_date]})
        attendance_df = pd.concat([attendance_df, new_record], ignore_index=True)

        print(f"Attendance marked for {predicted_name}")

In [ ]:
import time
try:
    while True:
        # Open the camera
        cap = cv2.VideoCapture(0)

        # Capture one photo
        ret, frame = cap.read()
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), scaleFactor=2, minNeighbors=2)

        if len(faces) > 0:
            # Extract the first face
            x, y, w, h = faces[0]
            face_roi = rgb_frame[y:y+h, x:x+w]

            # Resize the face image to match the model input size
            face_resized = cv2.resize(face_roi, (224, 224))

            # Normalize the pixel values
            face_resized = face_resized / 255.0

            # Reshape the image to match the model's expected shape
            face_resized = np.reshape(face_resized, (1, 224, 224, 3))

            # Make predictions using the model
            predictions = model.predict(face_resized)
            predicted_label = np.argmax(predictions)
            label_buffer.append(predicted_label)
            # Get the name associated with the predicted label
            predicted_name = label_encoder.inverse_transform([predicted_label])[0]
            print(f"predicted face is {predicted_name}")
            # Draw a green rectangle around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Display the name below the face
            cv2.putText(frame, predicted_name, (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            # Display the frame
            cv2.imshow('Face Recognition', frame)
            # Wait for user input
            key = cv2.waitKey(1)

            # If the key pressed is 's', add to attendance
            if key == ord('s'):
               # Mark attendance based on user input
                mark_attendance(predictions)
            elif key == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                break
            else:
                continue



except Exception as e:
    print(f"An error occurred: {e}")

finally:
    cap.release()
    # Close all OpenCV windows
    cv2.destroyAllWindows()

In [ ]:
attendance_df.head()
attendance_df.to_csv(csv_file_path, index=False)